In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# Load the dataset
file_path = 'sepsis_diab_pt_all_new1.xlsx'  # Update if needed
# Load all sheets
sheets = pd.ExcelFile(file_path)
sheet_names = sheets.sheet_names
print("Available Sheets:", sheet_names)

# Load individual sheets
admission_data = sheets.parse('sepsis_pt_all_admission details')
lab_events = sheets.parse('sepsis_lab_events')
microbiology_events = sheets.parse('microbiology events')
prescription_data = sheets.parse('prescriptoin')
poe_data = sheets.parse('poe')

print("admission_data shape", admission_data.shape)
print("lab_events shape", lab_events.shape)
print("microbiology_events", microbiology_events.shape)
print("prescription_data", prescription_data.shape)
print("poe_data",poe_data.shape)

In [ ]:
#admission_data
print("admission_data info",admission_data[['subject_id','hadm_id','admission_type','edhours','hospital_expire_flag','dx_1_code','sepsis_flag','drg_code','diabetic_flag','gender','anchor_age']].info())
print("head of admission data", admission_data[['subject_id','hadm_id','admission_type','edhours','hospital_expire_flag','dx_1_code','sepsis_flag','drg_code','diabetic_flag','gender','anchor_age']].head())

admission_data['edhours']=admission_data['edhours'].fillna(0)
admission_data['anchor_age']=admission_data['anchor_age'].fillna(admission_data['anchor_age'].mean())
admission_data['age_group'] = pd.cut(admission_data['anchor_age'], bins=[0, 18, 40, 65, 120],labels=['Child', 'Young Adult', 'Adult', 'Senior'])

#admission data cleaning
admission_data = admission_data.drop_duplicates(subset='hadm_id')

admission_data[['subject_id','hadm_id','admission_type','edhours','hospital_expire_flag','dx_1_code','sepsis_flag','drg_code','diabetic_flag','gender','anchor_age','age_group',"LOS"]].info()

In [ ]:
admission_data['gender'].value_counts()

In [ ]:
admission_data['gender']=admission_data['gender'].apply(lambda x:1 if x=="M" else 0) 

In [ ]:
admission_data['gender'].value_counts()

In [ ]:
#analysing prescription data
print("prescription_data info", prescription_data.info())
print("prescription_data head", prescription_data.head(5))
prescription_data = prescription_data.drop_duplicates(subset=['hadm_id', 'drug'])
prescription_summary = prescription_data.groupby('hadm_id')['drug'].apply(lambda x: '|'.join(x.unique()))
print("prescription_data summary df", prescription_summary.head(5))

In [ ]:
## Microbiology events
print("microbiology_events info", microbiology_events.info())
print("microbiology_events head", microbiology_events.head(3))      
microbiology_events['hadm_id']=microbiology_events['hadm_id'].fillna(microbiology_events['hadm_id.1'])
microbiology_events['hadm_id']=microbiology_events['hadm_id'].astype('int')
microbiology_events = microbiology_events.dropna(subset=['hadm_id', 'spec_type_desc'])
microbiology_summary = microbiology_events.groupby('hadm_id')['spec_type_desc'].apply(lambda x: '|'.join(x.unique()))
microbiology_flags = microbiology_events['spec_type_desc'].str.get_dummies(sep=',').groupby(microbiology_events['hadm_id']).max()
print("microbiology_summary",microbiology_summary.head(2))
print("microbiology_summary",microbiology_summary.head(2))

In [ ]:
## POE data
print("poe_data info",poe_data.info())
print("poe_data head",poe_data.head())

poe_data['order_type'] = poe_data['order_type'].str.strip()
poe_summary = poe_data.groupby('hadm_id')['order_type'].apply(lambda x: '|'.join(x.unique()))
poe_counts = poe_data.groupby(['hadm_id', 'order_type']).size().unstack(fill_value=0)
print("poe_summary info",poe_summary.info())
print("poe_summary head",poe_summary.head())
print("poe_counts info",poe_counts.info())
print("poe_counts head",poe_counts.head())

In [ ]:
admission_data['drg_code']=admission_data['drg_code'].astype(str)
admission_data['dx_1_code']=admission_data['dx_1_code'].astype(str)

# --- Step 2: Encode Features ---
## Categorical encoding
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
admission_encoded = pd.DataFrame(
    ohe.fit_transform(admission_data[['admission_type', 'drg_code', 'dx_1_code','gender','age_group']]),
    columns=ohe.get_feature_names_out())
## Numerical scaling
scaler = StandardScaler()
admission_data['scaled_edhours'] = scaler.fit_transform(admission_data[['edhours']])
admission_data['scaled_los'] = scaler.fit_transform(admission_data[['LOS']])

admission_data_ohc = pd.concat(
    [
        admission_data[['hadm_id', 'gender', 'age_group', 'scaled_edhours', 'scaled_los','LOS']].reset_index(drop=True),
        admission_encoded.reset_index(drop=True)
    ],
    axis=1
)

# --- Step 3: Merge Data ---

merged_data = admission_data_ohc.merge(
    prescription_summary, on='hadm_id', how='left'
).merge(
    microbiology_summary, on='hadm_id', how='left'
).merge(
    microbiology_flags, on='hadm_id', how='left'
).merge(
    poe_counts, on='hadm_id', how='left'
)

In [ ]:
merged_data.head(5)

In [ ]:
merged_data.info(verbose=True)

In [ ]:
merged_data.isnull().sum()

In [ ]:
print(list(merged_data.columns))

In [ ]:
merged_data.isnull().sum()

In [ ]:
merged_data[['ABSCESS', 'ASPIRATE', 'BILE', 'BIOPSY', 'BLOOD CULTURE', 'BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)', 'BONE MARROW', 'BONE MARROW - CYTOGENETICS', 'BRONCHIAL WASHINGS', 'BRONCHOALVEOLAR LAVAGE', 'Blood (CMV AB)', 'Blood (EBV)', 'Blood (LYME)', 'Blood (Malaria)', 'Blood (Toxo)', 'CATHETER TIP-IV', 'CSF;SPINAL FLUID', 'DIALYSIS FLUID', 'DIRECT ANTIGEN TEST FOR VARICELLA-ZOSTER VIRUS', 'Direct Antigen Test for Herpes Simplex Virus Types 1 & 2', 'FECAL SWAB', 'FLUID', 'FLUID RECEIVED IN BLOOD CULTURE BOTTLES', 'FOOT CULTURE', 'FOREIGN BODY', 'IMMUNOLOGY', 'Immunology (CMV)', 'Influenza A/B by DFA', 'Isolate', 'JOINT FLUID', 'MRSA SCREEN', 'Mini-BAL', 'OTHER', 'PERITONEAL FLUID', 'PLEURAL FLUID', 'POSTMORTEM CULTURE', 'Rapid Respiratory Viral Screen & Culture', 'SEROLOGY/BLOOD', 'SKIN SCRAPINGS', 'SPUTUM', 'STOOL', 'STOOL (RECEIVED IN TRANSPORT SYSTEM)', 'SWAB', 'Staph aureus swab', 'THROAT CULTURE', 'THROAT FOR STREP', 'TISSUE', 'URINE', 'VIRAL CULTURE: R/O CYTOMEGALOVIRUS', 'XXX', 'ADT orders', 'Blood Bank', 'Cardiology', 'Consults', 'Critical Care', 'General Care', 'Hemodialysis', 'IV therapy', 'Lab', 'Medications', 'Neurology', 'Nutrition', 'Radiology', 'Respiratory', 'TPN']]=merged_data[['ABSCESS', 'ASPIRATE', 'BILE', 'BIOPSY', 'BLOOD CULTURE', 'BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)', 'BONE MARROW', 'BONE MARROW - CYTOGENETICS', 'BRONCHIAL WASHINGS', 'BRONCHOALVEOLAR LAVAGE', 'Blood (CMV AB)', 'Blood (EBV)', 'Blood (LYME)', 'Blood (Malaria)', 'Blood (Toxo)', 'CATHETER TIP-IV', 'CSF;SPINAL FLUID', 'DIALYSIS FLUID', 'DIRECT ANTIGEN TEST FOR VARICELLA-ZOSTER VIRUS', 'Direct Antigen Test for Herpes Simplex Virus Types 1 & 2', 'FECAL SWAB', 'FLUID', 'FLUID RECEIVED IN BLOOD CULTURE BOTTLES', 'FOOT CULTURE', 'FOREIGN BODY', 'IMMUNOLOGY', 'Immunology (CMV)', 'Influenza A/B by DFA', 'Isolate', 'JOINT FLUID', 'MRSA SCREEN', 'Mini-BAL', 'OTHER', 'PERITONEAL FLUID', 'PLEURAL FLUID', 'POSTMORTEM CULTURE', 'Rapid Respiratory Viral Screen & Culture', 'SEROLOGY/BLOOD', 'SKIN SCRAPINGS', 'SPUTUM', 'STOOL', 'STOOL (RECEIVED IN TRANSPORT SYSTEM)', 'SWAB', 'Staph aureus swab', 'THROAT CULTURE', 'THROAT FOR STREP', 'TISSUE', 'URINE', 'VIRAL CULTURE: R/O CYTOMEGALOVIRUS', 'XXX', 'ADT orders', 'Blood Bank', 'Cardiology', 'Consults', 'Critical Care', 'General Care', 'Hemodialysis', 'IV therapy', 'Lab', 'Medications', 'Neurology', 'Nutrition', 'Radiology', 'Respiratory', 'TPN']].fillna(0)

In [ ]:
merged_data.isnull().sum()

In [ ]:
merged_data.info(verbose=True)

In [ ]:

# Final Feature Matrix (Exclude 'hadm_id' for modeling)
#feature_matrix = merged_data.drop(columns=['hadm_id'], errors='ignore')

# Export prepared data
merged_data.to_csv('RL_clinicalpathwayprepared_data_1.csv', index=False)

In [ ]:
#Action mapping

In [ ]:
unique_drugs=prescription_data['drug'].unique().tolist()
unique_microbiology_events=microbiology_events['spec_type_desc'].unique().tolist()
unique_poe_list=poe_data['order_type'].unique().tolist()

In [ ]:
# Initialize empty action_metadata dictionary
action_metadata = {}

# Assign categories to each action
for drug in unique_drugs:
    action_metadata[drug] = "drug"

for event in unique_microbiology_events:
    action_metadata[event] = "test"

for poe in unique_poe_list:
    action_metadata[poe] = "poe"

# Print the resulting action_metadata for verification
print(action_metadata)


In [ ]:
all_actions = unique_drugs + unique_microbiology_events + unique_poe_list

In [ ]:
all_actions

In [ ]:
action_map = {index: action for index, action in enumerate(all_actions)}

In [ ]:
print("Action Map:")
print(action_map)


In [ ]:
#merged_data.info(verbose=True)

In [ ]:
los_max = merged_data["LOS"].max() 
data=merged_data.drop(columns=['hadm_id','age_group','drug','spec_type_desc'])

In [ ]:
env = ClinicalEnvironment(data, action_map, los_max)

In [ ]:
data.head(2)

In [ ]:
#data.info(verbose=True)

In [ ]:
import numpy as np
import pandas as pd

class ClinicalEnvironment:
    def __init__(self, data, action_map, los_max, action_metadata):
        """
        Initialize the clinical environment.

        Parameters:
        - data: Prepared dataset with one-hot encoded features for drugs, lab tests, and POEs.
        - action_map: A mapping of action indices to clinical actions (drugs, lab tests, orders).
        - los_max: Maximum LOS in the dataset (used for normalization).
        """
        self.data = data
        self.action_map = action_map
        self.los_max = los_max
        self.current_index = 0
        self.done = False 
        self.action_metadata = action_metadata


    def reset(self):
        """
        Reset the environment by selecting a random admission.

        Returns:
        - Initial state as a NumPy array.
        """
        self.current_index = np.random.randint(0, len(self.data))
        self.done = False
        return self.data.iloc[self.current_index].drop(["LOS"]).values

    def step(self, action_index):
        """
        Take an action and transition to the next state.

        Parameters:
        - action_index: Index of the selected action.

        Returns:
        - next_state: Next state as a NumPy array.
        - reward: Reward for the taken action.
        - done: Whether the episode has ended.
        """
        row = self.data.iloc[self.current_index]
        action = self.action_map[action_index]
        
        reward = self._calculate_reward(row["LOS"], action)
        
        # Simulate a simple transition: State remains static.
        next_state = row.drop(["LOS"]).values + self._action_effect(action)
        
        # End the episode after one step (simplified setup).
        self.done = self.current_index >= len(self.data) - 1  # Replace with better condition
        return next_state, reward, self.done

    def _action_effect(self, action):
        """
        Simulate the effect of an action on the current state.
        """
        # Example: Return an array of zeros (no state change) or customize as needed.
        return np.zeros(len(self.data.columns) - 1)  # Adjust for actual features


    def _calculate_reward(self, los, action):
        category = self.action_metadata.get(action, "unknown")
        normalized_reward = (self.los_max - los) / self.los_max

        if category == "drug":
            normalized_reward += 0.1  # Example logic for drugs
        elif category == "test":
            normalized_reward += 0.05  # Example logic for tests
        elif category == "poe":
            normalized_reward -= 0.05  # Example logic for POEs

        return normalized_reward

        
    def get_initial_state_old(self, query_dict):
        """
        Convert a patient query dictionary into the initial state format
        based on the data columns of the environment.
        """
        # Ensure query_dict has the same keys as data columns minus "los"
        state = []
        print("len of columns inside initial state", len(self.data.columns))
        #for col in self.data.columns:
        #    if col == "los":
        #        continue  # Skip the target column
        #    state.append(query_dict.get(col, 0))  # Default to 0 if key is missing
        feature_columns = [col for col in self.data.columns if col != "LOS"]
        state = [query_dict.get(col, 0) for col in feature_columns]

        return np.array(state, dtype=np.float32)

    def get_initial_state(self, query_dict):
        """
        Convert a patient query dictionary into the initial state format
        based on the data columns of the environment.
        """
        # Ensure query_dict has the same keys as data columns minus "los"
        # Collect column names excluding "los"
        feature_columns = [col for col in self.data.columns if col != "LOS"]
        state = []
    
        print("len of columns inside initial state", len(feature_columns))
        for col in feature_columns:
            state.append(query_dict.get(col, 0))  # Default to 0 if key is missing
        return np.array(state, dtype=np.float32)


##DQN Model
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# Define the DQN model
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
#print(list(merged_data.columns))

In [ ]:
# Hyperparameters
state_size = len(data.columns) - 1  # Number of features
action_size = len(action_map)       # Number of possible actions
batch_size = 64
gamma = 0.99                        # Discount factor
epsilon = 1.0                       # Initial exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
target_update = 10                  # Frequency of target network update
episodes = 1000

# Initialize environment and networks
env = ClinicalEnvironment(data, action_map, los_max,action_metadata)
q_network = DQN(state_size, action_size)
target_network = DQN(state_size, action_size)
target_network.load_state_dict(q_network.state_dict())
optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

max_grad_norm = 1.0

def compute_target(q_values, target_q_values, actions, rewards, dones):
    """
    Compute the target Q-value using Double DQN logic.
    """
    best_actions = torch.argmax(q_values, dim=1)  # Actions selected by Q-network
    max_target_q = target_q_values.gather(1, best_actions.unsqueeze(1)).squeeze(1)
    return rewards + (1 - dones) * gamma * max_target_q


In [ ]:
q_network

In [ ]:
max_steps = 100  # Maximum steps per episode
memory = deque(maxlen=2000)
best_reward = float('-inf')

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False
    steps = 0

    while not done and steps < max_steps:
        steps += 1
        action = select_action(state, epsilon)
        next_state, reward, done = env.step(action)
        total_reward += reward

        memory.append((state, action, reward, next_state, done))
        state = next_state

        # Skip training if memory is insufficient
        if len(memory) < batch_size:
            continue

        # Sample and train
        batch = random.sample(memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.long)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)

        q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze()
        with torch.no_grad():
            target_q_values = rewards + gamma * (1 - dones) * target_network(next_states).max(1)[0]

        loss = loss_fn(q_values, target_q_values)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(q_network.parameters(), max_norm=1.0)
        optimizer.step()

    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    if episode % target_update == 0:
        target_network.load_state_dict(q_network.state_dict())

    if total_reward > best_reward:
        best_reward = total_reward
        torch.save(q_network.state_dict(), "best_q_network.pth")

    print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.2f}")

print("Training Complete!")


In [ ]:
torch.save(q_network.state_dict(), "dqn_clinical_pathway1.pth")

In [ ]:
q_network.load_state_dict(torch.load("dqn_clinical_pathway1.pth"))
q_network.eval()

In [ ]:
def recommend_pathway(env, model, query_dict):
    # Convert query_dict to the initial state format
    state = env.get_initial_state(query_dict)
    #print("State vector shape:", len(state))  # Length of the state vector
    #print("Number of columns in input data:", len(state.columns) - 1)  # Exclude 'los'
    #print("Number of columns in data:", len(env.data.columns) - 1)  # Exclude 'los'
    #print("Columns in data:", list(env.data.columns))
    #print("QNetwork input_dim:", model.fc1.in_features)  # Should match len(state)
    #print("QNetwork output_dim:", model.fc3.out_features)  # Should match len(env.action_map)
    
    recommendations = []
    done = False
    
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        print("State tensor shape:", state_tensor.shape)
        with torch.no_grad():
            action = torch.argmax(model(state_tensor)).item()
        next_state, reward, done = env.step(action)
        recommendations.append({
            "action": env.action_map[action],
            "reward": reward
        })
        state = next_state
    
    return recommendations

def recommend_pathway_other(env, model, query_dict, max_steps=100):
    state = torch.FloatTensor(env.get_initial_state(query_dict)).unsqueeze(0)
    pathway = []
    
    for step in range(max_steps):
        q_values = model(state)
        print(f"Q-values: {q_values}")
        action_index = torch.argmax(q_values).item()
        action_name = env.action_map[action_index]
        next_state, reward, done = env.step(action_index)
        pathway.append({"action": action_name, "reward": reward})
        
        #if done:
        #    break
        state = torch.FloatTensor(next_state).unsqueeze(0)
    
    return pathway

In [ ]:
data.info(2)

In [ ]:
query_dict = {
    #"gender": 1,  # Encoded
    #"age": 56,
    "drg_code_871": 1,
    #"age_group_Senior": 1,  # Encoded
    #"ed_hours": 3.5
}

In [ ]:
recommendations = recommend_pathway(env, q_network, query_dict)

In [ ]:
recommendations